In [1]:
import scipp as sc
import scippneutron as scn
from ess import loki, sans
from scippneutron.tof.conversions import elastic_wavelength, elastic_Q, beamline
from scipp.constants import m_n, h

In [2]:
def to_bin_edges(d, dim):
    centers = d.coords[dim].copy()
    del d.coords[dim]
    first = 1.5*centers[dim, 0] - 0.5*centers[dim, 1]
    last = 1.5*centers[dim, -1] - 0.5*centers[dim, -2]
    bulk = 0.5 * (centers[dim, 1:] + centers[dim, :-1])
    edges = sc.concat([first, bulk], dim)
    edges = sc.concat([edges, last], dim)
    d.coords[dim] = edges

In [3]:
def get_wavelength_independent_factor(l2, l_collimation, r1, r2, dr):
      
    inv_l3 = (l_collimation + l2) / (l_collimation * l2)
    
    # Terms in Mildner and Carpenter equation.
    # See https://docs.mantidproject.org/nightly/algorithms/TOFSANSResolutionByPixel-v1.html
    a1 = (r1/l_collimation)*(r1/l_collimation) * 3.0
    a2 = (r2*inv_l3)*(r2*inv_l3) * 3.0
    a3 = (dr/l2) * (dr/l2) 
    
    wav_independent = (4.0 * sc.constants.pi / 12) * (a1 + a2 + a3) 
    return wav_independent, a1, a2, a3
    

In [4]:
def generate_gaussian(sigma, term_name, num_points):
    """
    Generates gaussian function based on given sigma.
    Currently for visualization purposes. 
    In the future it may be used for numerical integration of Mildner-Carpenter terms 
    """
    import scipy.stats as stats
    xpoints = sc.linspace('x', -3*sigma, 3*sigma, num=num_points)
    pdf_result = stats.norm.pdf(xpoints.values,loc=0,scale=sigma.value)
    gauss_data = sc.Dataset(
    data={
        term_name:sc.array(dims=['x'], values=pdf_result)}, 
    coords={
        'x':sc.array(dims=['x'], values=xpoints.values)} )
    return gauss_data

In [5]:
def get_sigma_q_value(data, d_lam, lam, wav_independent, moderatorValue, q, l1, l2):

     
    #sigModerator = Moderator time spread (microseconds) as afunction of wavelength (Angstroms)
    sig_moderator = moderatorValue * 3.9560 / (1000.0 * (l1 + l2));
    sig_moderator.unit = sc.units.angstrom
    
    #TODO: replace with q from cooridnate transformations 
    theta = scn.two_theta(data) 
    q_sq = 4.0 * sc.constants.pi * sc.sin(0.5*theta) * sc.reciprocal(lam)
    q_sq *= q_sq
    
    std_dev_lam_sq = q_sq 
    std_dev_lam_sq *= (d_lam * d_lam)/12 + sig_moderator * sig_moderator
    std_dev_lam_sq *= sc.reciprocal(lam * lam)
    
    std_dev_lam_independent = sc.DataArray((wav_independent * sc.reciprocal(lam * lam) ),
                    coords={'wavelength':std_dev_lam_sq.coords['wavelength']}) 
   
    dq_sq = sc.sqrt(std_dev_lam_independent + std_dev_lam_sq)
    return dq_sq   


In [6]:
def q_resolution(wavelength_bins, moderator, data, q):
    
    l_collimation = 4.0*sc.units.m
    r2 = 0.004*sc.units.m
    r1 = 0.01*sc.units.m
    dr = 0.008*sc.units.m
    
    d_lam = wavelength_bins['wavelength', 1:] - wavelength_bins['wavelength', :-1] # bin widths
    lam = sc.midpoints(wavelength_bins)
    l2 = scn.L2(data)
    l1 = scn.L1(data)
    
    wav_independent, a1, a2, a3 = get_wavelength_independent_factor(l2, l_collimation, r1, r2, dr)
    
    to_bin_edges(moderator, 'wavelength')
    moderatorValue = sc.rebin(moderator, 'wavelength', wavelength_bins)
    
    dq_sq = get_sigma_q_value(data, d_lam, lam, wav_independent, moderatorValue, q, l1, l2)
    
    return a1, a2, a3, dq_sq 

In [7]:
# Estimate qresolution function

wavelength_bins = sc.linspace(dim='wavelength', start=2.0, stop=16.0, num=141, unit='angstrom')

path = 'SANS2D_data'
moderator_file = 'ModeratorStdDev_TS2_SANS_LETexptl_07Aug2015.txt'
moderator = loki.io.load_rkh_wav(filename=f'{path}/{moderator_file}')

#q resolution params
l_collimation = 4.0*sc.units.m
r2 = 0.004*sc.units.m
r1 = 0.01*sc.units.m
dr = 0.008*sc.units.m

# Using only one-fourth of the full spectra 245760 (reserved for first detector)
#spectrum_size =  245760//4

# Sample measurement
#sample = loki.io.load_sans2d(filename=f'{path}/SANS2D00063114.nxs',
#                             spectrum_size=spectrum_size)

#d - data is normalized and reduced data

#dq_sq = q_resolution(wavelength_bins, moderator, sample, l_collimation, r1, r2, dr)

#Dense example data - still needs to make it working for event data 
da = scn.data.tutorial_dense_data()

In [8]:
def two_theta(gravity, wavelength, incident_beam, scattered_beam):
    # Arbitrary internal convention: beam=z, gravity=y
    g = sc.norm(gravity)
    L2 = sc.norm(scattered_beam)
    y = sc.dot(scattered_beam, gravity) / g
    n = sc.cross(incident_beam, gravity)
    n /= sc.norm(n)
    x = sc.dot(scattered_beam, n)
    wavelength = sc.to_unit(wavelength, "m", copy=False)
    drop = g * m_n ** 2 / (2 * h ** 2) * wavelength ** 2 * L2 ** 2
    return sc.asin(sc.sqrt(x ** 2 + (y + drop) ** 2) / L2)

In [9]:
#Conversion of the data 
q_with_gravity = {**beamline(scatter=True),
                  **elastic_Q("tof")}
q_with_gravity["two_theta"] = two_theta
da.coords["gravity"] = sans.i_of_q.gravity_vector()
da.coords["tof"] = 0.5 * (da.coords["tof"]["tof", :-1] + da.coords["tof"]["tof", 1:])
data_q = da.transform_coords("Q", graph=q_with_gravity)


In [10]:
# Q binning
q_bins = sc.linspace(dim='Q', start=0.01, stop=0.6, num=141, unit='1/angstrom')
data_q = data_q.flatten(to="Q").hist(Q=q_bins)

In [20]:
data_q.plot(norm="log")

In [11]:
data_q.coords['Q']

<scipp.Variable> (Q: 141)    float64           [1/Å]  [0.01, 0.0142143, ..., 0.595786, 0.6]

In [12]:
a1, a2, a3, dq_sq = q_resolution(wavelength_bins, moderator, da, data_q.coords['Q'])

The 'bins' keyword argument and positional syntax for setting bin edges is deprecated. Use, e.g., 'sc.rebin(da, x=x_edges)'. See the documentation for details.


### Plotting individual resolution terms 
a2, a3 are generated per spectrum, so they are not easy to plot unless summed/collapsed?

In [13]:
gauss_data = generate_gaussian(a1, 'a1', 100)

In [14]:
gauss_data['a1'].plot()

### At the end we convert to Q and want to have dq in the function of Q (as a single number). 

In [15]:
dq_sq.coords['sample_position'] = da.coords['sample_position']
dq_sq.coords['source_position'] = da.coords['source_position']
dq_sq.coords['position'] = da.coords['position']
dq_sq.coords["gravity"] = sans.i_of_q.gravity_vector()

In [16]:
dq = dq_sq.transform_coords("Q", graph=q_with_gravity)

In [27]:
data_q.values

array([3.96382015e+06, 4.88722935e+06, 5.07247919e+06, 4.81532304e+06,
       4.39399085e+06, 3.91496561e+06, 3.45800449e+06, 3.02335670e+06,
       2.62877336e+06, 2.29613845e+06, 1.98955931e+06, 1.73581709e+06,
       1.51296387e+06, 1.32416115e+06, 1.15854122e+06, 1.01571268e+06,
       8.89397185e+05, 7.83915992e+05, 6.89225341e+05, 6.06450443e+05,
       5.37306603e+05, 4.74335883e+05, 4.22952789e+05, 3.74812007e+05,
       3.35164482e+05, 2.98459787e+05, 2.67467836e+05, 2.40879758e+05,
       2.15789402e+05, 1.95586824e+05, 1.76415294e+05, 1.59554564e+05,
       1.43679424e+05, 1.30265975e+05, 1.17691067e+05, 1.06403513e+05,
       9.57862120e+04, 8.66190448e+04, 7.79654913e+04, 7.05073405e+04,
       6.32696903e+04, 5.71802758e+04, 5.13576638e+04, 4.59680135e+04,
       4.18178963e+04, 3.72859723e+04, 3.37506157e+04, 3.00611727e+04,
       2.69701018e+04, 2.43564430e+04, 2.20799962e+04, 2.00224890e+04,
       1.84248863e+04, 1.68168485e+04, 1.53067703e+04, 1.40647135e+04,
      

In [32]:
dq['spectrum', 100].values

array([0.00551715, 0.00508172, 0.00470245, 0.00436976, 0.00407599,
       0.00381537, 0.00358299, 0.00337496, 0.00318779, 0.00301856,
       0.0028651 , 0.00272545, 0.00259805, 0.00248135, 0.00237404,
       0.00227518, 0.00218388, 0.00209941, 0.00202101, 0.00194801,
       0.00187994, 0.00181637, 0.00175693, 0.0017012 , 0.00164879,
       0.00159947, 0.001553  , 0.00150919, 0.00146779, 0.00142854,
       0.00139135, 0.00135605, 0.00132255, 0.00129069, 0.0012603 ,
       0.00123132, 0.00120367, 0.00117728, 0.00115205, 0.00112786,
       0.00110468, 0.00108246, 0.00106116, 0.00104071, 0.00102101,
       0.00100207, 0.00098383, 0.00096629, 0.00094938, 0.00093304,
       0.00091727, 0.00090204, 0.00088734, 0.00087313, 0.00085936,
       0.00084602, 0.00083311, 0.00082061, 0.0008085 , 0.00079673,
       0.00078531, 0.00077422, 0.00076347, 0.00075302, 0.00074284,
       0.00073294, 0.00072332, 0.00071396, 0.00070485, 0.00069596,
       0.0006873 , 0.00067886, 0.00067065, 0.00066263, 0.00065

In [18]:
dq['spectrum',100].plot()

In [36]:
dq_copy = dq

In [42]:
dq_copy.values = dq.values/data_q.values

In [43]:
dq_copy

<scipp.DataArray>
Dimensions: Sizes[spectrum:114688, Q:140, ]
Coordinates:
  Q                         float64           [1/Å]  (Q [bin-edge], spectrum)  [0.537638, 0.535708, ..., 0.0478506, 0.0480453]
Data:
                            float64           [1/Å]  (spectrum, Q)  [1.99675e-09, 1.48171e-09, ..., 8.57753e-07, 8.9751e-07]  [0, 0, ..., 0, 0]
Attributes:
  gravity                   vector3          [m/s^2]  ()  [(0, -9.80665, 0)]
  incident_beam             vector3              [m]  ()  [(0, 0, 25.3)]
  position                  vector3              [m]  (spectrum)  [(0.778, 0.130467, 29.8588), (0.775065, 0.130467, 29.8588), ..., (-0.569652, -0.0228657, 29.9533), (-0.572, -0.0228657, 29.9533)]
  sample_position           vector3              [m]  ()  [(0, 0, 25.3)]
  scattered_beam            vector3              [m]  (spectrum)  [(0.778, 0.130467, 4.55878), (0.775065, 0.130467, 4.55878), ..., (-0.569652, -0.0228657, 4.65328), (-0.572, -0.0228657, 4.65328)]
  source_position           vector3              [m]  ()  [(0, 0, 0)]
  two_theta                 float64            [rad]  (Q [bin-edge], spectrum)  [0.171345, 0.170728, ..., 0.121926, 0.122423]
  wavelength                float64             [Å]  (Q [bin-edge])  [2, 2.1, ..., 15.9, 16]

In [46]:
dq_copy.mean('spectrum').plot()